# LightGBM - Complete Guide

## 📚 Learning Objectives
- Understand LightGBM and its advantages over XGBoost
- Implement LightGBM for regression and classification
- Master categorical feature handling
- Optimize training speed and memory usage
- Compare LightGBM with XGBoost

## 🎯 What is LightGBM?

**LightGBM (Light Gradient Boosting Machine)** is a gradient boosting framework developed by Microsoft that uses tree-based learning algorithms.

### Key Innovations:
1. **Leaf-wise Growth**: Grows trees leaf-wise (best-first) instead of level-wise
2. **Histogram-based Algorithm**: Bins continuous features for faster training
3. **Native Categorical Support**: No need for one-hot encoding
4. **GOSS (Gradient-based One-Side Sampling)**: Faster training
5. **EFB (Exclusive Feature Bundling)**: Reduces dimensionality

### LightGBM vs XGBoost:

| Feature | LightGBM | XGBoost |
|---------|----------|----------|
| **Speed** | ⚡ Faster | Slower |
| **Memory** | 💾 Lower | Higher |
| **Accuracy** | Similar | Similar |
| **Categorical** | ✅ Native | ❌ Needs encoding |
| **Tree Growth** | Leaf-wise | Level-wise |
| **Large Datasets** | ✅ Better | Good |

### When to Use LightGBM:
✅ Large datasets (>10K rows)  
✅ Many categorical features  
✅ Need for speed  
✅ Memory constraints  
✅ High-dimensional data  

In [ ]:
# Install LightGBM if not already installed
# !pip install lightgbm

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.datasets import load_breast_cancer
import time
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"LightGBM version: {lgb.__version__}")

## Part 1: LightGBM for Regression
### 1️⃣ Load and Prepare Data

In [ ]:
# Load California Housing dataset
df = pd.read_csv('../../Linear Regression/data/dataset.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData types:")
print(df.dtypes)

df.head()

In [ ]:
# Prepare features and target
target_col = 'median_house_value'
X = df.drop(columns=[target_col])
y = df[target_col]

# LightGBM can handle categorical features natively!
# Mark categorical columns
categorical_features = ['ocean_proximity'] if 'ocean_proximity' in X.columns else []

# Convert to category dtype for LightGBM
for col in categorical_features:
    X[col] = X[col].astype('category')

print(f"\nCategorical features: {categorical_features}")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

### 2️⃣ Basic LightGBM Model

In [ ]:
# Create and train basic LightGBM model
lgbm_basic = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    num_leaves=31,  # LightGBM specific parameter
    random_state=42,
    n_jobs=-1,
    verbose=-1  # Suppress warnings
)

# Measure training time
print("Training LightGBM model...")
start_time = time.time()

lgbm_basic.fit(
    X_train, y_train,
    categorical_feature=categorical_features  # Specify categorical features
)

training_time = time.time() - start_time
print(f"✅ Training complete in {training_time:.2f} seconds!")

# Make predictions
y_train_pred = lgbm_basic.predict(X_train)
y_test_pred = lgbm_basic.predict(X_test)

# Evaluate
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\n📊 Basic LightGBM Performance:")
print(f"\nTraining Metrics:")
print(f"  RMSE: ${train_rmse:,.2f}")
print(f"  R²:   {train_r2:.4f}")
print(f"\nTest Metrics:")
print(f"  RMSE: ${test_rmse:,.2f}")
print(f"  R²:   {test_r2:.4f}")
print(f"\nOverfitting Check: {train_r2 - test_r2:.4f}")

### 3️⃣ Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgbm_basic.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🔍 Feature Importance:")
print(feature_importance)

# Visualize feature importance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot
feature_importance.plot(x='feature', y='importance', kind='barh', ax=ax1, 
                       color='lightgreen', edgecolor='black')
ax1.set_xlabel('Importance Score', fontsize=12)
ax1.set_ylabel('Features', fontsize=12)
ax1.set_title('Feature Importance (Split)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Built-in LightGBM plot
lgb.plot_importance(lgbm_basic, ax=ax2, importance_type='gain', max_num_features=10)
ax2.set_title('Feature Importance (Gain)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 4️⃣ Hyperparameter Tuning

#### Key LightGBM Parameters:

**Core Parameters:**
- `num_leaves`: Max number of leaves (default: 31)
- `max_depth`: Maximum tree depth (-1 = no limit)
- `learning_rate`: Step size shrinkage (0.01-0.3)
- `n_estimators`: Number of boosting rounds

**Learning Control:**
- `min_child_samples`: Minimum data in a leaf (default: 20)
- `subsample`: Fraction of data for training (0.5-1.0)
- `colsample_bytree`: Fraction of features (0.5-1.0)

**Regularization:**
- `reg_alpha`: L1 regularization
- `reg_lambda`: L2 regularization
- `min_split_gain`: Minimum gain to split

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_grid = {
    'num_leaves': [15, 31, 63],
    'max_depth': [5, 7, 10, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5],
    'min_child_samples': [10, 20, 30]
}

print("🔍 Starting hyperparameter tuning...")
print("This may take a few minutes...\n")

lgbm_random = RandomizedSearchCV(
    LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
    param_distributions=param_grid,
    n_iter=20,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

lgbm_random.fit(X_train, y_train)

print(f"\n🏆 Best Parameters: {lgbm_random.best_params_}")
print(f"Best CV Score (RMSE): ${np.sqrt(-lgbm_random.best_score_):,.2f}")

In [ ]:
# Evaluate tuned model
best_lgbm = lgbm_random.best_estimator_

y_train_pred_tuned = best_lgbm.predict(X_train)
y_test_pred_tuned = best_lgbm.predict(X_test)

train_rmse_tuned = np.sqrt(mean_squared_error(y_train, y_train_pred_tuned))
test_rmse_tuned = np.sqrt(mean_squared_error(y_test, y_test_pred_tuned))
train_r2_tuned = r2_score(y_train, y_train_pred_tuned)
test_r2_tuned = r2_score(y_test, y_test_pred_tuned)

print(f"\n📊 Tuned LightGBM Performance:")
print(f"\nTraining Metrics:")
print(f"  RMSE: ${train_rmse_tuned:,.2f}")
print(f"  R²:   {train_r2_tuned:.4f}")
print(f"\nTest Metrics:")
print(f"  RMSE: ${test_rmse_tuned:,.2f}")
print(f"  R²:   {test_r2_tuned:.4f}")

# Compare with basic model
print(f"\n📈 Improvement:")
print(f"  RMSE: ${test_rmse - test_rmse_tuned:,.2f} better")
print(f"  R²:   {test_r2_tuned - test_r2:.4f} better")

### 5️⃣ Early Stopping and Callbacks

In [ ]:
# Train with early stopping using callbacks
lgbm_early = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Create evaluation set
eval_set = [(X_train, y_train), (X_test, y_test)]

# Fit with callbacks
lgbm_early.fit(
    X_train, y_train,
    eval_set=eval_set,
    eval_metric='rmse',
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)

print(f"\nBest iteration: {lgbm_early.best_iteration_}")
print(f"Best score: {lgbm_early.best_score_}")

## Part 2: LightGBM for Classification
### 6️⃣ Binary Classification Example

In [ ]:
# Load breast cancer dataset
cancer = load_breast_cancer()
X_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y_cancer = pd.Series(cancer.target, name='diagnosis')

print(f"Classification dataset shape: {X_cancer.shape}")
print(f"\nClass distribution:")
print(y_cancer.value_counts())

# Split data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cancer, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
)

In [ ]:
# Train LightGBM classifier
lgbm_clf = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgbm_clf.fit(X_train_c, y_train_c)

# Predictions
y_pred_c = lgbm_clf.predict(X_test_c)
y_pred_proba_c = lgbm_clf.predict_proba(X_test_c)[:, 1]

# Evaluate
accuracy = accuracy_score(y_test_c, y_pred_c)
roc_auc = roc_auc_score(y_test_c, y_pred_proba_c)

print(f"\n📊 LightGBM Classification Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(y_test_c, y_pred_c, target_names=cancer.target_names))

### 7️⃣ LightGBM vs XGBoost Comparison

In [ ]:
# Import XGBoost for comparison
from xgboost import XGBRegressor

# Prepare data for fair comparison
X_encoded = X.copy()
if 'ocean_proximity' in X_encoded.columns:
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    X_encoded['ocean_proximity'] = le.fit_transform(X_encoded['ocean_proximity'])

X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# Define models with similar parameters
models = {
    'LightGBM': LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),
    'XGBoost': XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        n_jobs=-1
    )
}

# Compare training time and performance
comparison_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Measure training time
    start_time = time.time()
    model.fit(X_train_enc, y_train_enc)
    training_time = time.time() - start_time
    
    # Measure prediction time
    start_time = time.time()
    y_pred = model.predict(X_test_enc)
    prediction_time = time.time() - start_time
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test_enc, y_pred))
    r2 = r2_score(y_test_enc, y_pred)
    
    comparison_results[name] = {
        'Training Time (s)': training_time,
        'Prediction Time (s)': prediction_time,
        'RMSE': rmse,
        'R²': r2
    }
    
    print(f"  Training time: {training_time:.2f}s")
    print(f"  Prediction time: {prediction_time:.4f}s")
    print(f"  RMSE: ${rmse:,.2f}")
    print(f"  R²: {r2:.4f}")

# Display comparison
comparison_df = pd.DataFrame(comparison_results).T
print("\n📊 LightGBM vs XGBoost Comparison:")
print(comparison_df)

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training time
comparison_df['Training Time (s)'].plot(kind='bar', ax=axes[0, 0], color=['lightgreen', 'orange'], edgecolor='black')
axes[0, 0].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Time (seconds)')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Prediction time
comparison_df['Prediction Time (s)'].plot(kind='bar', ax=axes[0, 1], color=['lightgreen', 'orange'], edgecolor='black')
axes[0, 1].set_title('Prediction Time Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Time (seconds)')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# RMSE
comparison_df['RMSE'].plot(kind='bar', ax=axes[1, 0], color=['lightgreen', 'orange'], edgecolor='black')
axes[1, 0].set_title('RMSE Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('RMSE ($)')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# R²
comparison_df['R²'].plot(kind='bar', ax=axes[1, 1], color=['lightgreen', 'orange'], edgecolor='black')
axes[1, 1].set_title('R² Score Comparison', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 📊 Key Takeaways

### LightGBM Advantages:
✅ **Faster Training**: 2-10x faster than XGBoost  
✅ **Lower Memory**: More memory efficient  
✅ **Native Categorical**: No encoding needed  
✅ **Better for Large Data**: Handles millions of rows  
✅ **Leaf-wise Growth**: Can achieve better accuracy  

### Best Practices:
1. **Use categorical features natively** - Don't one-hot encode
2. **Start with num_leaves=31** - Good default
3. **Use early stopping** - Prevents overfitting
4. **Monitor max_depth** - Prevent too deep trees
5. **Tune num_leaves and max_depth together**

### Hyperparameter Tuning Strategy:
1. **Fix learning_rate** at 0.1
2. **Tune num_leaves** (15, 31, 63, 127)
3. **Tune max_depth** (5, 7, 10, -1)
4. **Tune min_child_samples** (10, 20, 30)
5. **Add regularization** (reg_alpha, reg_lambda)
6. **Lower learning_rate** and increase n_estimators

### LightGBM vs XGBoost:

**Choose LightGBM when:**
- Large datasets (>100K rows)
- Many categorical features
- Speed is critical
- Memory is limited

**Choose XGBoost when:**
- Small datasets (<10K rows)
- Need maximum stability
- More mature ecosystem needed

### Common Pitfalls:
❌ **Overfitting with leaf-wise growth** - Monitor carefully  
❌ **Too many leaves** - Can overfit quickly  
❌ **Not using categorical features** - Missing key advantage  
❌ **Ignoring min_child_samples** - Important for regularization  

### Advanced Features:
1. **Custom objectives** - Define your own loss function
2. **Dart mode** - Dropout for trees
3. **GOSS** - Gradient-based sampling
4. **Feature bundling** - Automatic dimensionality reduction

### Next Steps:
1. Try **CatBoost** for comparison
2. Implement **custom metrics**
3. Use **SHAP** for interpretability
4. Optimize for **production deployment**
5. Experiment with **dart** and **goss** modes